In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
from torchsummary import summary

In [4]:
import numpy as np

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
## computes output shape for both convolutions and pooling layers
def output_shape(in_dim,stride,padding,kernel,dilation=1):
    out_dim = np.floor((in_dim + 2*padding - dilation*(kernel-1)-1)/stride+1).astype(int)
    return out_dim

def output_shape_transpose(in_dim,stride,padding,kernel,output_padding, dilation=1):
    out_dim = (in_dim-1)*stride-2*padding+dilation*(kernel-1)+output_padding+1
    return out_dim

In [7]:
output_shape(np.arange(5),stride=1,padding=0,kernel=1)

array([0, 1, 2, 3, 4])

In [8]:
def get_dilation(out_dim,in_dim,stride,padding,kernel,output_padding):
    dilation = np.floor((out_dim-(in_dim-1)*stride+2*padding-output_padding-1)/(kernel-1))
    new_dim  = output_shape_transpose(in_dim,stride,padding,kernel,output_padding, dilation)
    if new_dim == out_dim:
        pass
    else:
        output_padding = (out_dim-new_dim)
    return dilation, output_padding

def get_output_padding(in_dim,out_dim, stride,padding,kernel,dilation=1):
    return out_dim-(in_dim-1)*stride+2*padding-dilation*(kernel-1)-1

In [9]:
class Reshape(nn.Module):
    def __init__(self, shape):
        super(Reshape, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)

In [ ]:
params['dim'] == '1D':

In [50]:
class ConvEncoder(nn.Module):
    def __init__(self, params, nparams):
        super(ConvEncoder, self).__init__()
        if params['dim'] == '1D':
            self.conv = nn.Conv1d
            self.pool = nn.AdaptiveMaxPool1d#nn.MaxPool1d
            self.N    = 1
        elif params['dim'] == '2D':
            self.conv = nn.Conv2d
            self.pool = nn.AdaptiveMaxPool2d
            self.N    = 2
        else:
            raise Exception("Invalid data dimensionality (must be either 1D or 2D).")
            
        self.model = nn.ModuleList()
    
        current_channels   = params['input_c']
        current_dim        = params['input_dim']
        self.out_dims      = []
        
        for ii in range(nparams['n_layers']):
            
            conv = self.conv(current_channels, nparams['out_channels'][ii], nparams['kernel_sizes'][ii], nparams['strides'][ii], nparams['paddings'][ii])
            self.out_dims.append(current_dim)
            self.model.append(conv)
            
            current_channels =  nparams['out_channels'][ii]
            current_dim      =  output_shape(current_dim, nparams['strides'][ii], nparams['paddings'][ii],nparams['kernel_sizes'][ii])
            
            gate = getattr(nn, nparams['activations'][ii])()
            self.model.append(gate)
            
            pool = self.pool([current_dim//nparams['scale_facs'][ii]]*self.N)
            self.model.append(pool)
            
            current_dim = current_dim//nparams['scale_facs'][ii]

        self.final_dim = current_dim
        self.final_c   = current_channels
        
        self.model.append(nn.Flatten())
        current_shape = current_channels*current_dim**self.N
        self.model.append(nn.Linear(current_shape,params['latent_dim']))

            
    def forward(self, x):
        for i, l in enumerate(self.model):
            x = l(x)
        return x

In [67]:
class ConvDecoder(nn.Module):
    def __init__(self, params, nparams, out_dims, final_dim, final_c):
        super(ConvDecoder, self).__init__()
        
        if params['dim'] == '1D':
            self.conv = nn.ConvTranspose1d
            self.N    = 1
        elif params['dim'] == '2D':
            self.conv = nn.ConvTranspose2d
            self.N    = 2
        else:
            raise Exception("Invalid data dimensionality (must be either 1D or 2D).")
        
        self.pool  = nn.Upsample
        
        self.model = nn.ModuleList()
        
        final_shape = final_c*final_dim**self.N
    
        self.model.append(nn.Flatten())
        self.model.append(nn.Linear(params['latent_dim'],final_shape))

        if params['dim'] == '1D':
            self.model.append(Reshape((-1, final_c,final_dim)))
        else:
            self.model.append(Reshape((-1, final_c,final_dim,final_dim)))
                              
        current_dim      = final_dim
        current_channels = final_c
            
        for jj in range(1,nparams['n_layers']+1):
            ii = nparams['n_layers'] - jj 
            gate = getattr(nn, nparams['activations'][ii])()
            self.model.append(gate)
                  
            upsample    = nn.Upsample(scale_factor=nparams['scale_facs'][ii])
            self.model.append(upsample)
            current_dim = current_dim*nparams['scale_facs'][ii]
                              
            output_padding = get_output_padding(current_dim,out_dims[ii],nparams['strides'][ii],nparams['paddings'][ii],nparams['kernel_sizes'][ii],dilation=1)
            conv           = self.conv(current_channels, nparams['out_channels'][ii], kernel_size=nparams['kernel_sizes'][ii], stride=nparams['strides'][ii], padding=nparams['paddings'][ii], output_padding=output_padding)
            self.model.append(conv)
            current_channels = out_channels[ii]
            current_dim      = output_shape_transpose(current_dim, stride=nparams['strides'][ii], padding=nparams['paddings'][ii],kernel=nparams['kernel_sizes'][ii],output_padding=output_padding)
                
        conv = self.conv(current_channels, 1, kernel_size=1, stride=1)
        self.model.append(conv)
        
    def forward(self, x):
        for i, l in enumerate(self.model):
            x = l(x)
        return x

In [73]:
class Autoencoder(nn.Module):
    def __init__(self, params, network_params):
        super(Autoencoder, self).__init__()
        
        if params['encoder_type']=='conv':
            self.encoder = ConvEncoder
        elif params['encoder_type']=='fc':
            self.encoder = FCEncoder
        else:
            raise Exception('invalid encoder type')
            
        if params['decoder_type']=='conv':
            self.decoder = ConvDecoder
        elif params['decoder_type']=='fc':
            self.decoder = FCDecoder
        else:
            raise Exception('invalid decoder type')
        
        self.encoder=self.encoder(params, network_params)
        self.decoder=self.decoder(params, network_params, self.encoder.out_dims, self.encoder.final_dim, self.encoder.final_c)
        
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [82]:
n_layers     = 3
out_channels = [2,4,8]
kernel_sizes = [2,4,2]
### stride,padding,kernel; [1,0,1] is identity
#pooling_layers = [[1,0,1], [1,0,2]]
scale_facs   = [2,2,1] 
paddings     = [0,0,0]
strides      = [2,4,2]
dim          = '1D'
activations  = ['ReLU', 'ReLU','ReLU']
latent_dim   = 4
input_c      = 1 
input_dim    = 1000
encoder_type = 'conv'
decoder_type = 'conv'

In [83]:
general_params      = {'input_c': input_c, 'input_dim': input_dim, 'latent_dim': latent_dim, 'encoder_type': encoder_type, 'decoder_type': decoder_type, 'dim': dim}

conv_network_params = {'n_layers': n_layers, 'out_channels': out_channels, 'kernel_sizes': kernel_sizes, 'scale_facs': scale_facs, 'paddings': paddings,'strides': strides, 'activations': activations}

In [84]:
AE = Autoencoder(general_params, conv_network_params)

In [85]:
AE.to(device)

Autoencoder(
  (encoder): ConvEncoder(
    (model): ModuleList(
      (0): Conv1d(1, 2, kernel_size=(2,), stride=(2,))
      (1): ReLU()
      (2): AdaptiveMaxPool1d(output_size=[250])
      (3): Conv1d(2, 4, kernel_size=(4,), stride=(4,))
      (4): ReLU()
      (5): AdaptiveMaxPool1d(output_size=[31])
      (6): Conv1d(4, 8, kernel_size=(2,), stride=(2,))
      (7): ReLU()
      (8): AdaptiveMaxPool1d(output_size=[15])
      (9): Flatten(start_dim=1, end_dim=-1)
      (10): Linear(in_features=120, out_features=4, bias=True)
    )
  )
  (decoder): ConvDecoder(
    (model): ModuleList(
      (0): Flatten(start_dim=1, end_dim=-1)
      (1): Linear(in_features=4, out_features=120, bias=True)
      (2): Reshape()
      (3): ReLU()
      (4): Upsample(scale_factor=1.0, mode=nearest)
      (5): ConvTranspose1d(8, 8, kernel_size=(2,), stride=(2,), output_padding=(1,))
      (6): ReLU()
      (7): Upsample(scale_factor=2.0, mode=nearest)
      (8): ConvTranspose1d(8, 4, kernel_size=(4,), stri

In [87]:
data =  torch.randn(1,1,1000).to('cuda')

In [88]:


res = AE.forward(data)

In [89]:
res.shape

torch.Size([1, 1, 1000])